### QTL mapping with tensorQTL
The V8 *cis*-eQTLs and *cis*-sQTLs were mapped using [FastQTL](https://github.com/francois-a/fastqtl). This notebook provides the equivalent commands for tensorQTL, which contains additional built-in commands, including for mapping conditionally independent eQTLs.

#### Required input files
The VCF can be obtained from [AnVIL](https://app.terra.bio/#workspaces/anvil-datastorage/AnVIL_GTEx_V8_hg38)
The following files are needed for tensorQTL:
```
gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_WGS_VCF_files/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.MAF01.bed
gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_WGS_VCF_files/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.MAF01.bim
gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_WGS_VCF_files/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.MAF01.fam
```
The expression and covariate matrices can be obtained from the [GTEx Portal](https://gtexportal.org/home/datasets), specifically the following archives (which need to be unpacked):
```
GTEx_Analysis_v8_eQTL_expression_matrices.tar
GTEx_Analysis_v8_eQTL_covariates.tar.gz
```

In [ ]:
import pandas as pd
import tensorqtl
from tensorqtl import genotypeio, cis, post

In [ ]:
# load VCF
plink_prefix_path = '/resources/vcfs/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.MAF01'
pr = genotypeio.PlinkReader(plink_prefix_path)
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

In [ ]:
# Example: mapping eQTLs for 'Brain_Cortex'
tissue_id = 'Brain_Cortex'
phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed('/resources/GTEx_Analysis_v8_eQTL_expression_matrices/{}.v8.normalized_expression.bed.gz'.format(tissue_id))
covariates_df = pd.read_csv('/resources/GTEx_Analysis_v8_eQTL_covariates/{}.v8.covariates.txt'.format(tissue_id), sep='\t', index_col=0).T

In [ ]:
# 1) compute permutations and define eGenes
egenes_df = cis.map_cis(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df)
post.calculate_qvalues(egenes_df, fdr=0.05, qvalue_lambda=0.85)

In [ ]:
# 2) compute nominal associations for all variant-gene pairs
cis.map_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df,
                covariates_df, tissue_id, output_dir='.')

In [ ]:
# 3) compute significant variant-gene pairs
pairs_df = post.get_significant_pairs(egenes_df, '{}'.format(tissue_id))

In [ ]:
# 4) compute conditionally independent eQTLs
indep_df = cis.map_independent(genotype_df, variant_df, egenes_df,
                               phenotype_df, phenotype_pos_df, covariates_df)